# Google Cloud Platform - Using Machine Learning APIs  

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

I am hiding my key in a file which will not be seen in gitup.  What you need to do is replace my code below with:
```
APIKEY="xxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your API key
```
<b> Note: Make sure you generate an API Key and replace the value above. The sample key will not work.</b>

In [68]:
with open('MyStuff/key.txt') as r:
  APIKEY=r.read()

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Translate API </li>
<li> Google Cloud Vision API </li>
<li> Google Natural Language API </li>
<li> Google Cloud Speech API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [69]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /usr/local/envs/py3env/lib/python3.5/site-packages (10.0.1)
proto-google-cloud-error-reporting-v1beta1 0.15.3 has requirement oauth2client<4.0dev,>=2.0.0, but you'll have oauth2client 4.1.2 which is incompatible.
google-cloud-bigquery 0.28.0 has requirement requests>=2.18.0, but you'll have requests 2.9.1 which is incompatible.
google-cloud-spanner 0.29.0 has requirement requests<3.0dev,>=2.18.4, but you'll have requests 2.9.1 which is incompatible.
gapic-google-cloud-datastore-v1 0.15.3 has requirement oauth2client<4.0dev,>=2.0.0, but you'll have oauth2client 4.1.2 which is incompatible.
proto-google-cloud-datastore-v1 0.90.4 has requirement oauth2client<4.0dev,>=2.0.0, but you'll have oauth2client 4.1.2 which is incompatible.
datalab 1.1.2 has requirement google-api-python-client==1.6.2, but you'll have google-api-python-client 1.6.6 which is incompatible.
datalab 1.1.2 has requirement httplib2==0.10.3, but you'll have httplib2 0.11.3 which is i

In [70]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/envs/py3env/lib/python3.5/site-packages (1.6.6)
Requirement not upgraded as not directly required: oauth2client<5.0.0dev,>=1.5.0 in /usr/local/envs/py3env/lib/python3.5/site-packages (from google-api-python-client) (4.1.2)
Requirement not upgraded as not directly required: six<2dev,>=1.6.1 in /usr/local/envs/py3env/lib/python3.5/site-packages (from google-api-python-client) (1.11.0)
Requirement not upgraded as not directly required: httplib2<1dev,>=0.9.2 in /usr/local/envs/py3env/lib/python3.5/site-packages (from google-api-python-client) (0.11.3)
Requirement not upgraded as not directly required: uritemplate<4dev,>=3.0.0 in /usr/local/envs/py3env/lib/python3.5/site-packages/uritemplate-3.0.0-py3.5.egg (from google-api-python-client) (3.0.0)
Requirement not upgraded as not directly required: pyasn1-modules>=0.0.5 in /usr/local/envs/py3env/lib/python3.5/site-packages (from oauth2client<5.0.0dev,>=1.5.0->google-api-py

## Invoke Translate API

[Google Cloud Translation](https://cloud.google.com/translate/docs/) documentation.  I know a lot has gone on here - see [my LSTM notebook](https://nbviewer.jupyter.org/github/jeffreyrnorton/Notebooks_MachineLearning/blob/master/DeepNetsWithKeras_ANN_LSTM.ipynb) where I trained a Seq2Seq LSTM network to do translation on a relatively small vocabulary.

Also note that this is a service.  The translation is not happening on the VM running the notebook, but is running as a service.  This is where we start seeing the true power of cloud compute!

In [71]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='fr', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> est-ce vraiment si simple?
amazing technology -> technologie étonnante
wow -> sensationnel


That is really cool - how would a Gallego (a person from Galicia in the Northwest corner of Spain) say it?

In [72]:
outputs = service.translations().list(source='en', target='gl', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> é realmente tan fácil?
amazing technology -> tecnoloxía sorprendente
wow -> wow


## Invoke Vision API

The [Vision API](https://cloud.google.com/vision/docs/) can work off an image in Cloud Storage or embedded directly into a POST message. I'll use Cloud Storage and do OCR on this image: <img src="https://storage.googleapis.com/cloud-training-demos/vision/sign2.jpg" width="200" />.  That photograph is from http://www.publicdomainpictures.net/view-image.php?image=15842.

In [73]:
# Running Vision API
import base64
IMAGE="gs://cloud-training-demos/vision/sign2.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{'textAnnotations': [{'boundingPoly': {'vertices': [{'y': 104, 'x': 152}, {'y': 104, 'x': 1082}, {'y': 658, 'x': 1082}, {'y': 658, 'x': 152}]}, 'locale': 'zh', 'description': '请您爱护和保\n护卫生创建优\n美水环境\n'}, {'boundingPoly': {'vertices': [{'y': 104, 'x': 179}, {'y': 104, 'x': 321}, {'y': 243, 'x': 321}, {'y': 243, 'x': 179}]}, 'description': '请'}, {'boundingPoly': {'vertices': [{'y': 108, 'x': 329}, {'y': 108, 'x': 471}, {'y': 243, 'x': 471}, {'y': 243, 'x': 329}]}, 'description': '您'}, {'boundingPoly': {'vertices': [{'y': 108, 'x': 479}, {'y': 108, 'x': 775}, {'y': 243, 'x': 775}, {'y': 243, 'x': 479}]}, 'description': '爱护'}, {'boundingPoly': {'vertices': [{'y': 111, 'x': 786}, {'y': 111, 'x': 925}, {'y': 243, 'x': 925}, {'y': 243, 'x': 786}]}, 'description': '和'}, {'boundingPoly': {'vertices': [{'y': 104, 'x': 933}, {'y': 104, 'x': 1079}, {'y': 239, 'x': 1079}, {'y': 239, 'x': 933}]}, 'description': '保'}, {'boundingPoly': {'vertices': [{'y': 300, 'x': 165}, {'y': 300, 'x': 3

In [74]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']
print(foreignlang, foreigntext)

zh 请您爱护和保
护卫生创建优
美水环境



## Translate sign

In [75]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='en', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

请您爱护和保
护卫生创建优
美水环境
 -> Please love and protect hygiene to create a beautiful water environment


## More OCR with the Vision API
OCR intrigues me - it is actually quite difficult to do well and there are engines like Tesseract that aren't too bad.  So I want to try the engine with a POST and see if it can extract some English for me, but on a very difficult sign (text with skew):

<img src="https://get.pxhere.com/photo/road-highway-advertising-travel-sign-community-usa-landmark-street-sign-attraction-historic-tourism-signage-road-sign-illinois-history-66-traffic-sign-route-66-mother-road-odell-644464.jpg" width=600>

In [76]:
IMAGE="https://get.pxhere.com/photo/road-highway-advertising-travel-sign-community-usa-landmark-street-sign-attraction-historic-tourism-signage-road-sign-illinois-history-66-traffic-sign-route-66-mother-road-odell-644464.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'imageUri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 5,
                }]
            }],
        })
responses = request.execute(num_retries=5)
print(responses)

{'responses': [{'textAnnotations': [{'boundingPoly': {'vertices': [{'y': 646, 'x': 1039}, {'y': 646, 'x': 4861}, {'y': 3183, 'x': 4861}, {'y': 3183, 'x': 1039}]}, 'locale': 'en', 'description': 'ROUTE\nROADSIDE\nOO ATTRACTION\n1932 STANDARD OIL GAS STATION, ODELL, ILLINOIS\nThis restoration is a project of the Route 66 Association of\non the National Register of Historic Places.\n'}, {'boundingPoly': {'vertices': [{'y': 687, 'x': 1180}, {'y': 647, 'x': 2103}, {'y': 874, 'x': 2113}, {'y': 915, 'x': 1190}]}, 'description': 'ROUTE'}, {'boundingPoly': {'vertices': [{'y': 898, 'x': 2540}, {'y': 795, 'x': 4304}, {'y': 1175, 'x': 4326}, {'y': 1277, 'x': 2562}]}, 'description': 'ROADSIDE'}, {'boundingPoly': {'vertices': [{'y': 1363, 'x': 1039}, {'y': 1338, 'x': 2354}, {'y': 1962, 'x': 2366}, {'y': 1987, 'x': 1051}]}, 'description': 'OO'}, {'boundingPoly': {'vertices': [{'y': 1433, 'x': 2555}, {'y': 1390, 'x': 4783}, {'y': 1794, 'x': 4790}, {'y': 1837, 'x': 2563}]}, 'description': 'ATTRACTION'}

<img src="https://get.pxhere.com/photo/road-highway-advertising-travel-sign-community-usa-landmark-street-sign-attraction-historic-tourism-signage-road-sign-illinois-history-66-traffic-sign-route-66-mother-road-odell-644464.jpg" width=600>

Checking the end of the text with the sign - how did we do?  We missed one full line of text - so OCR still remains a difficult text, even for Google!

## Sentiment analysis with Language API

Let's evaluate the sentiment of some famous quotes using [Google Cloud Natural Language API](https://cloud.google.com/natural-language/docs/).

In [77]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
quotes = [
  'To succeed, you must have tremendous perseverance, tremendous will.',
  'It’s not that I’m so smart, it’s just that I stay with problems longer.',
  'Love is quivering happiness.',
  'Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.',
  'What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?',
  'When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. '
]
for quote in quotes:
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': quote
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

POLARITY=1 MAGNITUDE=0.9 for To succeed, you must have tremendous perseverance, tremendous will.
POLARITY=-1 MAGNITUDE=0.5 for It’s not that I’m so smart, it’s just that I stay with problems longer.
POLARITY=1 MAGNITUDE=0.9 for Love is quivering happiness.
POLARITY=1 MAGNITUDE=0.9 for Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.
POLARITY=1 MAGNITUDE=0.2 for What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?
POLARITY=-1 MAGNITUDE=0.4 for When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. 


In a [paper](https://arxiv.org/pdf/1010.3003.pdf) published in 2010 by Bollen et al, it was claimed that there was 87% correlation between tweets and the stock market.  In January 2013, the following *false* tweet was sent which [momentarily sent Serepta Therapeutics falling](http://fortune.com/2015/12/07/dataminr-hedge-funds-twitter-data/), but when investors realized the ruse, it quickly recovered.  Let's process the [tweet](http://kiddynamitesworld.com/the-sec-needs-to-arrest-some-people/).

"$SRPT FDA steps in as its 48 weeks results on Eteplirsen results are tainted and have been doctored they believeTrial papers seized by FDA."

In [78]:
quotes = [
  '$SRPT FDA steps in as its 48 weeks results on Eteplirsen results are tainted and have been doctored they believeTrial papers seized by FDA.'
]
for quote in quotes:
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': quote
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

POLARITY=-1 MAGNITUDE=0.9 for $SRPT FDA steps in as its 48 weeks results on Eteplirsen results are tainted and have been doctored they believeTrial papers seized by FDA.


And we see that this *is* a very negative statement - no wonder it impacted the market as Forbes relates.

<h2> Speech API </h2>

The [Speech API](https://cloud.google.com/speech-to-text/docs/) can work on streaming data, audio content encoded and embedded directly into the POST message, or on a file on Cloud Storage. Pass in this <a href="https://storage.googleapis.com/cloud-training-demos/vision/audio.raw">audio file</a> from Cloud Storage.

In [79]:
sservice = build('speech', 'v1beta1', developerKey=APIKEY)
response = sservice.speech().syncrecognize(
    body={
        'config': {
            'encoding': 'LINEAR16',
            'sampleRate': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()
print(response)

{'results': [{'alternatives': [{'transcript': 'how old is the Brooklyn Bridge', 'confidence': 0.98360395}]}]}


In [80]:
print(response['results'][0]['alternatives'][0]['transcript'])
print('Confidence=%f' % response['results'][0]['alternatives'][0]['confidence'])

how old is the Brooklyn Bridge
Confidence=0.983604


<h2> Clean up </h2>

Remember to delete the API key by visiting <a href="http://console.cloud.google.com/apis">API console</a>.

If necessary, commit all your notebooks to git.

If you are running Datalab on a Compute Engine VM or delegating to one, remember to stop or shut it down so that you are not charged.


## Challenge Exercise

Here are a few portraits from the Metropolitan Museum of Art, New York (they are part of a [BigQuery public dataset](https://bigquery.cloud.google.com/dataset/bigquery-public-data:the_met) ):

gs://gcs-public-data--met/14295/0.jpg  
<img src="https://raw.githubusercontent.com/jeffreyrnorton/Notebooks_MachineLearning/master/images/14295.jpg" width=400>

gs://gcs-public-data--met/15091/0.jpg  
<img src="https://raw.githubusercontent.com/jeffreyrnorton/Notebooks_MachineLearning/master/images/15091.jpg" width=400>

(Two given in the original assignment are not publically available and what good is it to tell you unhappy or happy when you can't see the photo?)

Use the Vision API to identify which of these images depict happy people and which ones depict unhappy people.

Hint: You will need to look for joyLikelihood and/or sorrowLikelihood from the response.

In [81]:
IMAGE="gs://gcs-public-data--met/14295/0.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcsImageUri': IMAGE
                    }
                },
                'features': [{
                    'type': 'FACE_DETECTION'
                }]
            }],
        })
responses = request.execute()
print(responses)

{'responses': [{'faceAnnotations': [{'headwearLikelihood': 'VERY_LIKELY', 'landmarkingConfidence': 0.40494683, 'blurredLikelihood': 'VERY_UNLIKELY', 'underExposedLikelihood': 'VERY_UNLIKELY', 'landmarks': [{'position': {'y': 1239.4186, 'x': 1657.5417, 'z': -0.00067241676}, 'type': 'LEFT_EYE'}, {'position': {'y': 1269.9839, 'x': 1709.117, 'z': 6.37109}, 'type': 'RIGHT_EYE'}, {'position': {'y': 1221.4691, 'x': 1648.5151, 'z': 3.180908}, 'type': 'LEFT_OF_LEFT_EYEBROW'}, {'position': {'y': 1238.1575, 'x': 1679.7795, 'z': -9.626153}, 'type': 'RIGHT_OF_LEFT_EYEBROW'}, {'position': {'y': 1251.854, 'x': 1703.0846, 'z': -6.6261573}, 'type': 'LEFT_OF_RIGHT_EYEBROW'}, {'position': {'y': 1266.6852, 'x': 1730.7343, 'z': 13.025247}, 'type': 'RIGHT_OF_RIGHT_EYEBROW'}, {'position': {'y': 1255.671, 'x': 1685.7417, 'z': -8.505504}, 'type': 'MIDPOINT_BETWEEN_EYES'}, {'position': {'y': 1286.2218, 'x': 1670.6033, 'z': -23.763466}, 'type': 'NOSE_TIP'}, {'position': {'y': 1302.7893, 'x': 1660.6646, 'z': -9.9

As a matter of interest - we can crop to the face with the following code:
```
xlow = ylow = 100000
xhigh = yhigh = -1
for point in responses['responses'][0]['faceAnnotations'][0]['boundingPoly']['vertices']:
    x = point['x']
    y = point['y']
    if x < xlow: xlow = x
    if y < ylow: ylow = y
    if x > xhigh: xhigh = x
    if y > yhigh: yhigh = y

from PIL import Image
import urllib.request

url = "https://raw.githubusercontent.com/jeffreyrnorton/Notebooks_MachineLearning/master/images/14295.jpg"
response = urllib.request.urlretrieve(url, "tmp/i.jpg")
img = Image.open("tmp/i.jpg")
img2 = img.crop((xlow, ylow, xhigh, yhigh))
img2.save("tmp/img2.jpg")
```

But of course, what we are really interested in are the emotions which we can print out.

In [82]:
annotation = responses['responses'][0]['faceAnnotations'][0]
print('Surprise: {}, Joy: {}, Sorrow: {}'.format(annotation['surpriseLikelihood'], annotation['joyLikelihood'], annotation['sorrowLikelihood']))

Surprise: VERY_UNLIKELY, Joy: LIKELY, Sorrow: VERY_UNLIKELY


<img src="https://raw.githubusercontent.com/jeffreyrnorton/Notebooks_MachineLearning/master/images/14295.jpg" width=400>

Now that we have explored this - let's write the code to process the other image very concisely and as a function where we assume we are always getting the image from a Google bucket.

In [83]:
def emotion_detector(apikey, image):
    vservice = build('vision', 'v1', developerKey=apikey)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcsImageUri': image
                    }
                },
                'features': [{
                    'type': 'FACE_DETECTION'
                }]
            }],
        })
    responses = request.execute()
    annotation = responses['responses'][0]['faceAnnotations'][0]
    return {'Surprise':annotation['surpriseLikelihood'],
            'Joy':annotation['joyLikelihood'],
            'Sorrow':annotation['sorrowLikelihood']}

IMAGE="gs://gcs-public-data--met/14295/0.jpg"
print(emotion_detector(APIKEY, IMAGE))



{'Surprise': 'VERY_UNLIKELY', 'Joy': 'LIKELY', 'Sorrow': 'VERY_UNLIKELY'}


<img src="https://raw.githubusercontent.com/jeffreyrnorton/Notebooks_MachineLearning/master/images/15091.jpg" width=400>

Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.